In [1]:
import visual_behavior_glm.src.GLM_params as glm_params
import visual_behavior_glm.src.GLM_analysis_tools as gat
import visual_behavior_glm.src.GLM_visualization_tools as gvt
from visual_behavior_glm.src.glm import GLM
import matplotlib.pyplot as plt
import visual_behavior.data_access.loading as loading
import visual_behavior.database as db
import plotly.express as px

import pandas as pd
import numpy as np
import os

import seaborn as sns

import visual_behavior.plotting as vbp

import plotly.express as px
from sklearn.cluster import KMeans
import umap

/home/dougo/Code/AllenSDK/allensdk/brain_observatory/behavior/behavior_ophys_api/behavior_ophys_nwb_api.py:9: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


In [2]:
%matplotlib notebook
%widescreen

applied a custom magic command to make full use of screen width
will only work if command is defined locally
replace with the following to replicate functionality: 
	from IPython.core.display import display, HTML
	display(HTML("<style>.container { width:100% !important; }</style>")


# Gather/organize data

## load the results summary for a single GLM version from mongo

In [3]:
glm_version = '4_L2_optimize_by_cell'
dropout_summary = gat.retrieve_results(search_dict = {'glm_version': glm_version}, results_type='summary')
dropout_summary.sample(5)

,_id,index,dropout,variance_explained,fraction_change_from_full,absolute_change_from_full,cell_specimen_id,ophys_experiment_id,ophys_session_id,behavior_session_id,container_id,project_code,container_workflow_state,experiment_workflow_state,session_name,session_type,equipment_name,date_of_acquisition,isi_experiment_id,specimen_id,sex,age_in_days,full_genotype,reporter_line,driver_line,imaging_depth,targeted_structure,published_at,super_container_id,cre_line,session_tags,failure_tags,exposure_number,model_outputs_available,location,glm_version,entry_time_utc
841887,5f223ff0a480a6bfb1e079db,31,beh_model,0.030512,0.031548,0.000933,1007039907,915136302,914163299,914741760,911248946,VisualBehaviorTask1B,container_qc,passed,20190731_464204_1imagesB,OPHYS_1_images_B,CAM2P.5,2019-07-31 16:32:28.000000,883781677,870720274,F,124.0,Slc17a7-IRES2-Cre/wt;Camk2a-tTA/wt;Ai93(TITL-G...,['Ai93(TITL-GCaMP6f)'],"['Camk2a-tTA', 'Slc17a7-IRES2-Cre']",375,VISp,NaN,870720274,Slc17a7-IRES2-Cre,NaN,NaN,0,1,Slc17a7_VISp_375,4_L2_optimize_by_cell,2020-07-31 18:08:38.897137
269812,5f2227791e0a039095095fd5,20,image3,0.026329,0.006341,0.000166,1028767628,887386947,887031077,887165432,1018027549,VisualBehaviorMultiscope,completed,passed,20190614_451787_6imagesB,OPHYS_6_images_B,MESO.1,2019-06-14 12:31:30.000000,846136902,837581585,M,135.0,Slc17a7-IRES2-Cre/wt;Camk2a-tTA/wt;Ai93(TITL-G...,['Ai93(TITL-GCaMP6f)'],"['Camk2a-tTA', 'Slc17a7-IRES2-Cre']",227,VISp,NaN,837581585,Slc17a7-IRES2-Cre,NaN,NaN,0,1,Slc17a7_VISp_227,4_L2_optimize_by_cell,2020-07-31 18:03:48.667144
565787,5f222d210a77a9012962bce2,14,image_expectation,0.060054,-0.010477,-0.000636,1017239466,992084627,991958444,991992557,1018027778,VisualBehaviorMultiscope,completed,passed,20191203_485152_Ophys1,OPHYS_1_images_A,MESO.1,2019-12-03 11:02:23.724167,949762677,935565389,M,145.0,Slc17a7-IRES2-Cre/wt;Camk2a-tTA/wt;Ai93(TITL-G...,['Ai93(TITL-GCaMP6f)'],"['Camk2a-tTA', 'Slc17a7-IRES2-Cre']",275,VISp,NaN,935565389,Slc17a7-IRES2-Cre,NaN,NaN,0,1,Slc17a7_VISp_275,4_L2_optimize_by_cell,2020-07-31 18:08:41.944448
915360,5f224560fad74c0a6b0b6843,0,Full,0.066076,0.000000,0.000000,962608413,808619543,808107961,808324301,803517529,VisualBehavior,container_qc,passed,20190104_425493_4imagesB,OPHYS_4_images_B,CAM2P.4,2019-01-14 17:59:54.000000,782229094,772629813,M,111.0,Slc17a7-IRES2-Cre/wt;Camk2a-tTA/wt;Ai93(TITL-G...,['Ai93(TITL-GCaMP6f)'],"['Camk2a-tTA', 'Slc17a7-IRES2-Cre']",375,VISp,NaN,772629813,Slc17a7-IRES2-Cre,NaN,NaN,0,1,Slc17a7_VISp_375,4_L2_optimize_by_cell,2020-07-31 18:16:17.990314
665327,5f22344810b86c4514ea2150,15,running,-0.020585,-0.025972,0.000549,962582477,825130141,824745199,824894721,812858755,VisualBehavior,container_qc,passed,20190218_431023_VISp_175um_3a,OPHYS_3_images_A,CAM2P.5,2019-02-18 19:29:08.000000,796874559,789992909,M,116.0,Slc17a7-IRES2-Cre/wt;Camk2a-tTA/wt;Ai93(TITL-G...,['Ai93(TITL-GCaMP6f)'],"['Camk2a-tTA', 'Slc17a7-IRES2-Cre']",175,VISp,NaN,789992909,Slc17a7-IRES2-Cre,NaN,NaN,1,1,Slc17a7_VISp_175,4_L2_optimize_by_cell,2020-07-31 18:15:22.711969


In [4]:
dropout_summary_pivoted = gat.build_pivoted_results_summary(results_summary=dropout_summary, cutoff=0.01, value_to_use='fraction_change_from_full')

In [5]:
behavior_model_summary = loading.get_behavior_model_summary_table()
behavior_model_summary.sample(10)

,behavior_session_id,ophys_session_id,ophys_experiment_id,project_code,session_type,equipment_name,date_of_acquisition,specimen_id,sex,age_in_days,full_genotype,reporter_line,driver_line,cre_line,donor_id,active,imaging_depth,container_id,image_set,trained_A,trained_B,stage,good,num_hits,num_fa,num_cr,num_miss,num_aborts,session_roc,lick_fraction,lick_fraction_1st,lick_fraction_2nd,lick_hit_fraction,lick_hit_fraction_1st,lick_hit_fraction_2nd,trial_hit_fraction,trial_hit_fraction_1st,trial_hit_fraction_2nd,task_dropout_index,task_only_dropout_index,timing_only_dropout_index,prior_bias,avg_weight_bias,avg_weight_bias_1st,avg_weight_bias_2nd,prior_omissions1,avg_weight_omissions1,avg_weight_omissions1_1st,avg_weight_omissions1_2nd,prior_task0,avg_weight_task0,avg_weight_task0_1st,avg_weight_task0_2nd,prior_timing1D,avg_weight_timing1D,avg_weight_timing1D_1st,avg_weight_timing1D_2nd,task_weight_index,task_weight_index_1st,task_weight_index_2nd,task_session,container_in_order,full_container,meso
82,922717464,922511481,[923094585],VisualBehaviorTask1B,OPHYS_1_images_B,CAM2P.4,2019-08-13 17:35:13.000000,879053662,M,119.0,Vip-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,['Ai148(TIT2L-GC6f-ICL-tTA2)'],['Vip-IRES-Cre'],Vip-IRES-Cre,879053649,True,175.0,920688952.0,B,False,True,1,True,112.0,7.0,28.0,120.0,343.0,0.870057,0.110845,0.152989,0.072926,0.242424,0.241722,0.243750,0.482759,0.912500,0.256579,-11.385398,-8.067904,-19.453302,0.044828,-2.361617,-2.066971,-2.626717,0.029198,0.575925,1.202221,0.012429,0.035617,2.507948,3.389757,1.714560,0.205206,1.831782,5.262374,-1.254812,0.676165,-1.872616,2.969372,False,True,False,NaN
85,920839478,920695792,[921024187 921024189 921024191 921024193 92102...,VisualBehaviorMultiscope,OPHYS_4_images_B,MESO.1,2019-08-09 12:55:56.000000,840544752,M,182.0,Vip-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,['Ai148(TIT2L-GC6f-ICL-tTA2)'],['Vip-IRES-Cre'],Vip-IRES-Cre,840544745,True,NaN,NaN,NaN,NaN,NaN,4,True,56.0,2.0,28.0,165.0,448.0,0.907376,0.114497,0.224858,0.015711,0.109589,0.090717,0.351351,0.253394,0.860000,0.076023,-14.181699,-4.310464,-18.492163,0.100589,-2.553967,-0.268762,-4.599492,0.040203,0.568408,1.303597,-0.089672,0.094383,2.454677,2.122642,2.751888,0.210199,0.471856,4.053688,-2.734301,1.982821,-1.931046,5.486189,False,NaN,NaN,True
192,962984229,962733051,[963397268],VisualBehaviorTask1B,OPHYS_4_images_A,CAM2P.5,2019-10-08 15:20:43.000000,912554454,F,133.0,Slc17a7-IRES2-Cre/wt;Camk2a-tTA/wt;Ai93(TITL-G...,['Ai93(TITL-GCaMP6f)'],"['Camk2a-tTA', 'Slc17a7-IRES2-Cre']",Slc17a7-IRES2-Cre,912554442,True,175.0,947838486.0,A,False,True,4,True,110.0,3.0,31.0,124.0,465.0,0.802318,0.138132,0.179438,0.098540,0.185497,0.204244,0.152778,0.470085,0.810526,0.237410,2.682155,-6.906565,-4.224410,0.065777,-2.101219,-1.726465,-2.460415,0.016107,0.712585,1.040295,0.398480,0.049606,2.189855,2.990912,1.422054,0.137397,0.153114,1.874774,-1.497072,2.036741,1.116138,2.919126,True,True,True,NaN
9,795742990,795625712,[795953296],VisualBehavior,OPHYS_4_images_B,CAM2P.4,2018-12-14 20:17:05.000000,744911458,F,152.0,Vip-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,['Ai148(TIT2L-GC6f-ICL-tTA2)'],['Vip-IRES-Cre'],Vip-IRES-Cre,744911447,True,175.0,791352433.0,B,True,False,4,True,95.0,6.0,23.0,113.0,467.0,0.840248,0.132291,0.184713,0.084633,0.167549,0.161804,0.178947,0.456731,0.802632,0.257576,-7.258538,-3.776737,-11.035275,0.089638,-2.118428,-1.371983,-2.797045,0.046016,0.412612,1.342339,-0.432633,0.042670,1.789163,2.322181,1.304581,0.117432,1.340900,2.922905,-0.097351,0.448264,-0.600725,1.401932,False,True,True,NaN
101,994710581,994689422,[994791582],VisualBehaviorTask1B,OPHYS_6_images_A,CAM2P.4,2019-12-16 17:00:21.000000,957212154,F,134.0,Vip-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,['Ai148(TIT2L-GC6f-ICL-tTA2)'],['Vip-IRES-Cre'],Vip-IRES-Cre,957212144,True,175.0,991913246.0,A,False,True,6,True,76.0,5.0,10.0,34.0,894.0,0.758169,0.232767,0.236183,0.229353,0.077079,0.076000,0.078189,0.690909,0.703704,0.678571,-9.022170,-0.749102,-9.7712

In [6]:
behavior_model_summary.query('ophys_session_id == 872592724')

,behavior_session_id,ophys_session_id,ophys_experiment_id,project_code,session_type,equipment_name,date_of_acquisition,specimen_id,sex,age_in_days,full_genotype,reporter_line,driver_line,cre_line,donor_id,active,imaging_depth,container_id,image_set,trained_A,trained_B,stage,good,num_hits,num_fa,num_cr,num_miss,num_aborts,session_roc,lick_fraction,lick_fraction_1st,lick_fraction_2nd,lick_hit_fraction,lick_hit_fraction_1st,lick_hit_fraction_2nd,trial_hit_fraction,trial_hit_fraction_1st,trial_hit_fraction_2nd,task_dropout_index,task_only_dropout_index,timing_only_dropout_index,prior_bias,avg_weight_bias,avg_weight_bias_1st,avg_weight_bias_2nd,prior_omissions1,avg_weight_omissions1,avg_weight_omissions1_1st,avg_weight_omissions1_2nd,prior_task0,avg_weight_task0,avg_weight_task0_1st,avg_weight_task0_2nd,prior_timing1D,avg_weight_timing1D,avg_weight_timing1D_1st,avg_weight_timing1D_2nd,task_weight_index,task_weight_index_1st,task_weight_index_2nd,task_session,container_in_order,full_container,meso
67,872646068,872592724,[873970521 873970523 873970525 873970527 87397...,VisualBehaviorMultiscope,OPHYS_1_images_A,MESO.1,2019-05-21 09:22:38.000000,837628436,M,123.0,Vip-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,['Ai148(TIT2L-GC6f-ICL-tTA2)'],['Vip-IRES-Cre'],Vip-IRES-Cre,837628429,True,NaN,NaN,NaN,NaN,NaN,1,True,59.0,2.0,26.0,118.0,541.0,0.876586,0.140852,0.208434,0.079755,0.096405,0.090698,0.10989,0.333333,0.735849,0.16129,-20.659579,-0.890258,-21.549837,0.160962,-2.852639,-0.646675,-4.846901,0.002018,0.902914,0.908795,0.897597,0.028468,1.298596,1.613424,1.013982,0.226578,2.576229,4.258479,1.055421,-1.277632,-2.645055,-0.041439,False,NaN,NaN,True


In [7]:
fig,ax=plt.subplots()
ax.hist(behavior_model_summary['task_dropout_index'])
ax.set_xlabel('task_dropout_index')

<IPython.core.display.Javascript object>

Text(0.5, 0, 'task_dropout_index')

## get the dominant dropout for each cell in each session

In [8]:
def get_dominant_dropout(row, cols_to_check):
    dominant_dropout = None
    dominant_dropout_value = 0
    for col in cols_to_check:
        if row[col] < dominant_dropout_value:
            dominant_dropout = col
            dominant_dropout_value = row[col]
    return dominant_dropout

cols_to_check = ['all-images','omissions','running','pupil']
dropout_summary_pivoted['dominant_dropout'] = dropout_summary_pivoted.apply(get_dominant_dropout, axis=1, args=[cols_to_check])

In [9]:
dropout_summary_pivoted['dominant_dropout'].unique()

array(['all-images', 'omissions', 'running', 'pupil', None], dtype=object)

## merge in task dropout index

In [10]:
dropout_summary_pivoted = dropout_summary_pivoted.merge(
    behavior_model_summary[['ophys_session_id','task_dropout_index']],
    left_on = 'ophys_session_id',
    right_on = 'ophys_session_id',
    how='left'
)

In [11]:
dropout_summary_pivoted.sample(10)

,identifier,Full,all-images,beh_model,change,correct_rejects,false_alarms,hits,image0,image1,image2,image3,image4,image5,image6,image7,image_expectation,intercept,misses,model_bias,model_omissions1,model_task0,model_timing1D,omissions,post_lick_bouts,post_licks,pre_lick_bouts,pre_licks,pupil,rewards,running,time,visual,cell_specimen_id,ophys_experiment_id,ophys_session_id,behavior_session_id,container_id,project_code,container_workflow_state,experiment_workflow_state,session_name,session_type,equipment_name,date_of_acquisition,isi_experiment_id,specimen_id,sex,age_in_days,full_genotype,reporter_line,driver_line,imaging_depth,targeted_structure,published_at,super_container_id,cre_line,session_tags,failure_tags,exposure_number,model_outputs_available,location,glm_version,dominant_dropout,task_dropout_index
13272,894727297_962602210,0.0,-0.645081,0.056919,0.000556,0.040429,2.976076e-02,0.010379,-0.009392,0.004675,0.003473,-0.047228,0.002686,0.004413,0.005462,0.002352,0.002183,0.005836,0.007438,-0.013802,0.001377,0.037061,0.050247,0.067000,0.004979,0.015527,0.058924,0.054487,0.000862,0.020935,-0.143440,0.024463,-0.583282,962602210,894727297,893937838,894083470,890047007,VisualBehavior,container_qc,passed,20190625_456916_6ImagesB,OPHYS_6_images_B,CAM2P.5,2019-06-25 15:37:48.000000,855988827,847074148,F,124.0,Slc17a7-IRES2-Cre/wt;Camk2a-tTA/wt;Ai93(TITL-G...,['Ai93(TITL-GCaMP6f)'],"['Camk2a-tTA', 'Slc17a7-IRES2-Cre']",175,VISp,NaN,847074148,Slc17a7-IRES2-Cre,NaN,NaN,0,1,Slc17a7_VISp_175,4_L2_optimize_by_cell,all-images,6.118085
322,789359614_962567653,0.0,-0.931996,0.041163,0.000074,0.006093,3.852452e-03,0.000801,-0.001374,-0.033905,-0.000343,-0.001085,-0.006735,-0.001674,-0.000919,-0.003645,-0.002216,0.006611,0.001473,0.014174,0.035980,0.025207,0.012025,0.007211,0.001288,0.001293,0.005713,0.005987,-0.010301,0.003325,0.002130,0.012157,-0.942982,962567653,789359614,789220000,789295700,814796558,VisualBehavior,container_qc,passed,20181130_416369_2P3_VISp_375_6,OPHYS_6_images_B,CAM2P.5,2018-11-30 23:28:37.000000,761346702,756577249,F,115.0,Slc17a7-IRES2-Cre/wt;Camk2a-tTA/wt;Ai93(TITL-G...,['Ai93(TITL-GCaMP6f)'],"['Camk2a-tTA', 'Slc17a7-IRES2-Cre']",375,VISp,NaN,756577249,Slc17a7-IRES2-Cre,NaN,"epilepsy_probability,z_drift_corr_um_diff",0,1,Slc17a7_VISp_375,4_L2_optimize_by_cell,all-images,2.574555
12731,891996193_962581660,0.0,-0.935273,0.010099,0.000082,0.000842,1.892548e-12,0.001233,0.000451,0.000321,0.000101,-0.000292,0.000394,0.000301,-0.049426,0.000276,0.000526,0.000852,0.000793,0.004725,-0.010909,-0.001851,0.004275,0.008302,0.000195,-0.000342,0.002630,0.001219,0.006793,-0.000172,-0.031261,-0.001434,-0.933305,962581660,891996193,891465652,891605682,902748650,VisualBehavior,container_qc,passed,20190620_456917_3imagesA,OPHYS_3_images_A,CAM2P.3,2019-06-20 17:30:46.000000,855685080,847076524,F,119.0,Slc17a7-IRES2-Cre/wt;Camk2a-tTA/wt;Ai93(TITL-G...,['Ai93(TITL-GCaMP6f)'],"['Camk2a-tTA', 'Slc17a7-IRES2-Cre']",375,VISp,NaN,847076524,Slc17a7-IRES2-Cre,NaN,NaN,0,1,Slc17a7_VISp_375,4_L2_optimize_by_cell,all-images,8.882868
379,789359614_962593127,0.0,-0.666953,0.011011,0.000033,0.001193,1.938943e-04,0.000235,-0.017069,-0.001707,-0.013315,-0.000110,0.000043,-0.000241,0.000076,0.000113,-0.000212,-0.000160,-0.001055,0.002594,0.003121,-0.000483,0.002880,-0.000157,0.000426,0.000284,0.007061,0.001760,-0.008619,0.001173,-0.040670,0.000387,-0.686558,962593127,789359614,789220000,789295700,814796558,VisualBehavior,container_qc,passed,20181130_416369_2P3_VISp_375_6,OPHYS_6_images_B,CAM2P.5,2018-11-30 23:28:37.000000,761346702,756577249,F,115.0,Slc17a7-IRES2-Cre/wt;Camk2a-tTA/wt;Ai93(TITL-G...,['Ai93(TITL-GCaMP6f)'],"['Camk2a-tTA', 'Slc17a7-IRES2-Cre']",375,VISp,NaN,756577249,Slc17a7-IRES2-Cre,NaN,"epilepsy_probability,z_drift_corr_um_diff",0,1,Slc17a7_VISp_375,4_L2_optimize_by_cell,all-images,2.574555
9673,880375092_962004200,0.0,0.051769,0.019265,0.001189,0.026150,2.558643e-02,0.014951,0.004148,0.009078,0.009736,0.012547,0.010105,0.009188,0.006325,

## add a `session_id` column with a numeric value for the session_type (to lump together sessions by order, regardless of image set)

In [12]:
def map_session_types(session_type):
    session_id = int(session_type[6:7])
    return session_id
dropout_summary_pivoted['session_id'] = dropout_summary_pivoted['session_type'].map(lambda st:map_session_types(st))

dropout_summary_pivoted

,identifier,Full,all-images,beh_model,change,correct_rejects,false_alarms,hits,image0,image1,image2,image3,image4,image5,image6,image7,image_expectation,intercept,misses,model_bias,model_omissions1,model_task0,model_timing1D,omissions,post_lick_bouts,post_licks,pre_lick_bouts,pre_licks,pupil,rewards,running,time,visual,cell_specimen_id,ophys_experiment_id,ophys_session_id,behavior_session_id,container_id,project_code,container_workflow_state,experiment_workflow_state,session_name,session_type,equipment_name,date_of_acquisition,isi_experiment_id,specimen_id,sex,age_in_days,full_genotype,reporter_line,driver_line,imaging_depth,targeted_structure,published_at,super_container_id,cre_line,session_tags,failure_tags,exposure_number,model_outputs_available,location,glm_version,dominant_dropout,task_dropout_index,session_id
0,1000743008_1003146172,0.0,-0.140223,-0.001848,0.000581,0.014875,0.008197,0.005280,-0.000634,0.003108,0.002692,0.001819,0.003621,-0.001186,-0.026248,0.002645,0.000947,0.000913,-0.001280,-0.001924,0.000501,-0.000538,0.000135,0.008544,0.007587,-0.005937,0.011424,0.014952,-0.074100,0.010710,-0.020992,-0.001004,-0.228425,1003146172,1000743008,1000580083,1000600602,999191979,VisualBehaviorTask1B,holding,passed,20200116_489056_1imagesB,OPHYS_1_images_B,CAM2P.3,2020-01-16 18:49:41.000000,962029605,951667115,M,165.0,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,['Ai148(TIT2L-GC6f-ICL-tTA2)'],['Sst-IRES-Cre'],275,VISp,NaN,951667115,Sst-IRES-Cre,NaN,NaN,0,1,Sst_VISp_275,4_L2_optimize_by_cell,all-images,-12.176364,1
1,1000743008_1003146180,0.0,-0.155019,0.004636,0.000530,0.020636,0.011811,0.003584,0.002961,0.003979,-0.007203,-0.007210,0.002283,-0.018611,0.002152,0.002657,0.000976,0.000081,0.006030,0.000728,0.001277,0.000856,0.002049,0.020436,0.001011,0.003103,0.031004,0.037226,0.005513,0.032333,-0.002625,0.000434,-0.252412,1003146180,1000743008,1000580083,1000600602,999191979,VisualBehaviorTask1B,holding,passed,20200116_489056_1imagesB,OPHYS_1_images_B,CAM2P.3,2020-01-16 18:49:41.000000,962029605,951667115,M,165.0,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,['Ai148(TIT2L-GC6f-ICL-tTA2)'],['Sst-IRES-Cre'],275,VISp,NaN,951667115,Sst-IRES-Cre,NaN,NaN,0,1,Sst_VISp_275,4_L2_optimize_by_cell,all-images,-12.176364,1
2,1000743008_1003146183,0.0,-0.210215,0.062499,0.000224,0.002435,0.001871,0.001834,0.001881,0.002758,-0.015761,0.000383,0.000302,-0.012279,0.008475,0.001058,-0.001232,-0.002156,0.000572,0.010619,0.029392,-0.001927,0.021750,-0.000316,-0.004966,0.002149,0.003036,0.008150,-0.057209,-0.000740,0.014037,0.004291,-0.319959,1003146183,1000743008,1000580083,1000600602,999191979,VisualBehaviorTask1B,holding,passed,20200116_489056_1imagesB,OPHYS_1_images_B,CAM2P.3,2020-01-16 18:49:41.000000,962029605,951667115,M,165.0,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,['Ai148(TIT2L-GC6f-ICL-tTA2)'],['Sst-IRES-Cre'],275,VISp,NaN,951667115,Sst-IRES-Cre,NaN,NaN,0,1,Sst_VISp_275,4_L2_optimize_by_cell,all-images,-12.176364,1
3,1000743008_1003146186,0.0,-0.368535,0.021722,0.000232,0.002308,0.006869,0.002990,-0.001212,-0.003161,0.000206,-0.049498,0.001615,-0.021775,0.000046,0.002221,-0.003134,0.003374,0.002284,0.014580,0.006673,0.005393,-0.001058,0.002760,-0.001018,0.000574,0.004443,0.004108,0.005936,0.011120,0.001867,0.002495,-0.435852,1003146186,1000743008,1000580083,1000600602,999191979,VisualBehaviorTask1B,holding,passed,20200116_489056_1imagesB,OPHYS_1_images_B,CAM2P.3,2020-01-16 18:49:41.000000,962029605,951667115,M,165.0,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,['Ai148(TIT2L-GC6f-ICL-tTA2)'],['Sst-IRES-Cre'],275,VISp,NaN,951667115,Sst-IRES-Cre,NaN,NaN,0,1,Sst_VISp_275,4_L2_optimize_by_cell,all-images,-12.176364,1
4,1000743008_1003146190,0.0,-0.089674,0.000823,0.000713,0.027508,0.019429,0.006967,0.006088,0.004045,-0.003198,-0.024926,0.003739,0.002454,-0.000550,0.008610,0.001467,0.000038,0.000181,-0.000054,0.001188,0.000657,-0.001699,0.031173,0.010330,0.005812,0.031873,0.030949,-0.019613,0.021468,0.005709,0.000490,-0.184349,1003146

In [13]:
dropout_summary_pivoted.columns

Index(['identifier', 'Full', 'all-images', 'beh_model', 'change',
       'correct_rejects', 'false_alarms', 'hits', 'image0', 'image1', 'image2',
       'image3', 'image4', 'image5', 'image6', 'image7', 'image_expectation',
       'intercept', 'misses', 'model_bias', 'model_omissions1', 'model_task0',
       'model_timing1D', 'omissions', 'post_lick_bouts', 'post_licks',
       'pre_lick_bouts', 'pre_licks', 'pupil', 'rewards', 'running', 'time',
       'visual', 'cell_specimen_id', 'ophys_experiment_id', 'ophys_session_id',
       'behavior_session_id', 'container_id', 'project_code',
       'container_workflow_state', 'experiment_workflow_state', 'session_name',
       'session_type', 'equipment_name', 'date_of_acquisition',
       'isi_experiment_id', 'specimen_id', 'sex', 'age_in_days',
       'full_genotype', 'reporter_line', 'driver_line', 'imaging_depth',
       'targeted_structure', 'published_at', 'super_container_id', 'cre_line',
       'session_tags', 'failure_tags', 'exposu

In [14]:
dropout_summary_pivoted.equipment_name.unique()

array(['CAM2P.3', 'CAM2P.4', 'CAM2P.5', 'MESO.1'], dtype=object)

## Turn some categorical columns into numerical columns

In [15]:
def make_categorical(df, column):
    df['{}_categorical'.format(column)] = pd.Categorical(df[column], ordered=True).codes

for column in ['cre_line','equipment_name','targeted_structure','session_id','dominant_dropout']:
    make_categorical(dropout_summary_pivoted, column)
    
dropout_summary_pivoted

,identifier,Full,all-images,beh_model,change,correct_rejects,false_alarms,hits,image0,image1,image2,image3,image4,image5,image6,image7,image_expectation,intercept,misses,model_bias,model_omissions1,model_task0,model_timing1D,omissions,post_lick_bouts,post_licks,pre_lick_bouts,pre_licks,pupil,rewards,running,time,visual,cell_specimen_id,ophys_experiment_id,ophys_session_id,behavior_session_id,container_id,project_code,container_workflow_state,experiment_workflow_state,session_name,session_type,equipment_name,date_of_acquisition,isi_experiment_id,specimen_id,sex,age_in_days,full_genotype,reporter_line,driver_line,imaging_depth,targeted_structure,published_at,super_container_id,cre_line,session_tags,failure_tags,exposure_number,model_outputs_available,location,glm_version,dominant_dropout,task_dropout_index,session_id,cre_line_categorical,equipment_name_categorical,targeted_structure_categorical,session_id_categorical,dominant_dropout_categorical
0,1000743008_1003146172,0.0,-0.140223,-0.001848,0.000581,0.014875,0.008197,0.005280,-0.000634,0.003108,0.002692,0.001819,0.003621,-0.001186,-0.026248,0.002645,0.000947,0.000913,-0.001280,-0.001924,0.000501,-0.000538,0.000135,0.008544,0.007587,-0.005937,0.011424,0.014952,-0.074100,0.010710,-0.020992,-0.001004,-0.228425,1003146172,1000743008,1000580083,1000600602,999191979,VisualBehaviorTask1B,holding,passed,20200116_489056_1imagesB,OPHYS_1_images_B,CAM2P.3,2020-01-16 18:49:41.000000,962029605,951667115,M,165.0,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,['Ai148(TIT2L-GC6f-ICL-tTA2)'],['Sst-IRES-Cre'],275,VISp,NaN,951667115,Sst-IRES-Cre,NaN,NaN,0,1,Sst_VISp_275,4_L2_optimize_by_cell,all-images,-12.176364,1,1,0,1,0,0
1,1000743008_1003146180,0.0,-0.155019,0.004636,0.000530,0.020636,0.011811,0.003584,0.002961,0.003979,-0.007203,-0.007210,0.002283,-0.018611,0.002152,0.002657,0.000976,0.000081,0.006030,0.000728,0.001277,0.000856,0.002049,0.020436,0.001011,0.003103,0.031004,0.037226,0.005513,0.032333,-0.002625,0.000434,-0.252412,1003146180,1000743008,1000580083,1000600602,999191979,VisualBehaviorTask1B,holding,passed,20200116_489056_1imagesB,OPHYS_1_images_B,CAM2P.3,2020-01-16 18:49:41.000000,962029605,951667115,M,165.0,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,['Ai148(TIT2L-GC6f-ICL-tTA2)'],['Sst-IRES-Cre'],275,VISp,NaN,951667115,Sst-IRES-Cre,NaN,NaN,0,1,Sst_VISp_275,4_L2_optimize_by_cell,all-images,-12.176364,1,1,0,1,0,0
2,1000743008_1003146183,0.0,-0.210215,0.062499,0.000224,0.002435,0.001871,0.001834,0.001881,0.002758,-0.015761,0.000383,0.000302,-0.012279,0.008475,0.001058,-0.001232,-0.002156,0.000572,0.010619,0.029392,-0.001927,0.021750,-0.000316,-0.004966,0.002149,0.003036,0.008150,-0.057209,-0.000740,0.014037,0.004291,-0.319959,1003146183,1000743008,1000580083,1000600602,999191979,VisualBehaviorTask1B,holding,passed,20200116_489056_1imagesB,OPHYS_1_images_B,CAM2P.3,2020-01-16 18:49:41.000000,962029605,951667115,M,165.0,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,['Ai148(TIT2L-GC6f-ICL-tTA2)'],['Sst-IRES-Cre'],275,VISp,NaN,951667115,Sst-IRES-Cre,NaN,NaN,0,1,Sst_VISp_275,4_L2_optimize_by_cell,all-images,-12.176364,1,1,0,1,0,0
3,1000743008_1003146186,0.0,-0.368535,0.021722,0.000232,0.002308,0.006869,0.002990,-0.001212,-0.003161,0.000206,-0.049498,0.001615,-0.021775,0.000046,0.002221,-0.003134,0.003374,0.002284,0.014580,0.006673,0.005393,-0.001058,0.002760,-0.001018,0.000574,0.004443,0.004108,0.005936,0.011120,0.001867,0.002495,-0.435852,1003146186,1000743008,1000580083,1000600602,999191979,VisualBehaviorTask1B,holding,passed,20200116_489056_1imagesB,OPHYS_1_images_B,CAM2P.3,2020-01-16 18:49:41.000000,962029605,951667115,M,165.0,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,['Ai148(TIT2L-GC6f-ICL-tTA2)'],['Sst-IRES-Cre'],275,VISp,NaN,951667115,Sst-IRES-Cre,NaN,NaN,0,1,Sst_VISp_275,4_L2_optimize_by_cell,all-images,-12.176364,1,1,0,1,0,0
4,1000743008_1003146190,0.0,-0.089674,0.000823,0.000713,0.027508,0.019429,0.006967,0.006088,0.004045,-0.003198,-0.024926,0.003739,0.002454,-0.000550,0.

## category == -1 represents None, set it to NaN

In [16]:
np.sort(dropout_summary_pivoted['dominant_dropout_categorical'].unique())

array([-1,  0,  1,  2,  3], dtype=int8)

In [17]:
dropout_summary_pivoted.loc[dropout_summary_pivoted.query('dominant_dropout_categorical == -1').index,'dominant_dropout_categorical'] = np.nan

## merge in some columns from the original response_summary

In [18]:
dominant_dropout_matrix = (
    dropout_summary_pivoted
    .drop_duplicates(['cell_specimen_id','session_id'])
    .pivot(index='cell_specimen_id',columns='session_id',values='dominant_dropout_categorical')
    .rename(columns={k:'session_{}'.format(k) for k in dropout_summary_pivoted['session_id'].unique()})
)

cols_to_merge = [
    'cell_specimen_id',
    'cre_line',
    'cre_line_categorical',
    'imaging_depth',
    'targeted_structure',
    'targeted_structure_categorical',
    'equipment_name_categorical',
    'task_dropout_index',
]
dominant_dropout_matrix = dominant_dropout_matrix.merge(
    dropout_summary_pivoted[cols_to_merge],
    left_on='cell_specimen_id',
    right_on='cell_specimen_id',
    how='left',
).set_index('cell_specimen_id')
dominant_dropout_matrix

,session_1,session_3,session_4,session_6,cre_line,cre_line_categorical,imaging_depth,targeted_structure,targeted_structure_categorical,equipment_name_categorical,task_dropout_index
cell_specimen_id,,,,,,,,,,,
817101743,NaN,NaN,NaN,1.0,Vip-IRES-Cre,2,175,VISp,1,1,-7.488484
817101771,0.0,0.0,NaN,3.0,Vip-IRES-Cre,2,175,VISp,1,1,-4.313725
817101771,0.0,0.0,NaN,3.0,Vip-IRES-Cre,2,175,VISp,1,1,-12.645683
817101771,0.0,0.0,NaN,3.0,Vip-IRES-Cre,2,175,VISp,1,1,-7.488484
817101837,0.0,1.0,NaN,3.0,Vip-IRES-Cre,2,175,VISp,1,1,-4.313725
...,...,...,...,...,...,...,...,...,...,...,...
1028894295,0.0,NaN,NaN,NaN,Vip-IRES-Cre,2,81,VISp,1,3,-20.659579
1028894317,2.0,NaN,NaN,NaN,Vip-IRES-Cre,2,81,VISp,1,3,-20.659579
1028894343,0.0,NaN,NaN,NaN,Vip-IRES-Cre,2,81,VISp,1,3,-20.659579


## define plot/heatmap parameters

In [19]:
# make sure there's a heatmap definition for every column, otherwise the column will plot without a heatmap!
cols_to_plot = [
    'session_1',
    'session_3',
    'session_4',
    'session_6',
    'cre_line_categorical',
    'targeted_structure_categorical',
    'imaging_depth',
    'equipment_name_categorical',
    'task_dropout_index'
]
heatmap_defs = [
    {
        'columns':['session_{}'.format(i) for i in [1,3,4,6]],
        'cbar_label':'dominant_dropout',
        'cbar_ticks':np.arange(0.5,4.5),
        'cbar_ticklabels':np.sort([c for c in dropout_summary_pivoted['dominant_dropout'].unique() if pd.notnull(c)]),
        'vmin':0,
        'vmax':4,
        'cmap':sns.color_palette("Dark2", 4),
    },
    {
        'columns':['cre_line_categorical'],
        'cbar_label':'cre_line',
        'cbar_ticks':[0,1,2],
        'cbar_ticklabels':np.sort(np.unique(dropout_summary_pivoted['cre_line'])),
        'vmin':-0.5,
        'vmax':2.5,
        'cmap':sns.color_palette("hls", 3),
    },
    {
        'columns':['equipment_name_categorical'],
        'cbar_label':'equipment name',
        'cbar_ticks':np.arange(len(np.unique(dropout_summary_pivoted['equipment_name']))),
        'cbar_ticklabels':np.sort(np.unique(dropout_summary_pivoted['equipment_name'])),
        'vmin':-0.5,
        'vmax':len(np.unique(dropout_summary_pivoted['equipment_name']))-0.5,
        'cmap':sns.color_palette("hls", len(np.unique(dropout_summary_pivoted['equipment_name']))),
    },
    {
        'columns':['targeted_structure_categorical'],
        'cbar_label':'targeted structure',
        'cbar_ticks':np.arange(len(dropout_summary_pivoted['targeted_structure_categorical'].unique())),
        'cbar_ticklabels':np.sort(np.unique(dropout_summary_pivoted['targeted_structure'])),
        'vmin':-0.5,
        'vmax':len(dropout_summary_pivoted['targeted_structure'].unique())-0.5,
        'cmap':sns.color_palette("hls", len(dropout_summary_pivoted['targeted_structure'].unique())),
    },
    {
        'columns':['imaging_depth'],
        'cbar_label':'imaging_depth',
        'cbar_ticks':[0,100,200,300,400],
        'vmin':0,
        'vmax':400,
        'cmap':'magma',
    },
    {
        'columns':['task_dropout_index'],
        'cbar_label':'task_dropout_index',
        'cbar_ticks':[-30,-20,-10,0,10,20,30],
        'vmin':-40,
        'vmax':40,
        'cmap':'RdBu',
    },
]

# plots

## Everything, sorted by cre-line, dropouts in sessions, depth

In [20]:
sort_by = [
    'cre_line_categorical',
    'session_1',
    'session_3',
    'session_4',
    'session_6',
    'imaging_depth'
]
data_to_plot = dominant_dropout_matrix[cols_to_plot].sort_values(by=sort_by)
fig, axes = vbp.make_multi_cmap_heatmap(
    data_to_plot, 
    heatmap_defs, 
    figsize=(10,12), 
    top_buffer=0, 
    bottom_buffer=0.1, 
    n_cbar_rows=6, 
    heatmap_div=0.9, 
)

<IPython.core.display.Javascript object>

## only cells that are matched across all sessions, same sorting as above

In [21]:
sort_by = [
    'cre_line_categorical',
    'session_1',
    'session_3',
    'session_4',
    'session_6',
    'imaging_depth'
]
data_to_plot = dominant_dropout_matrix[cols_to_plot].sort_values(by=sort_by).dropna()
fig, axes = vbp.make_multi_cmap_heatmap(
    data_to_plot, 
    heatmap_defs, 
    figsize=(10,12), 
    top_buffer=0, 
    bottom_buffer=0.1, 
    n_cbar_rows=6, 
    heatmap_div=0.9, 
)

<IPython.core.display.Javascript object>

In [22]:
sort_by = [
    'cre_line_categorical',
    'task_dropout_index',
    'session_1',
    'session_3',
    'session_4',
    'session_6',
    'imaging_depth'
]
data_to_plot = dominant_dropout_matrix[cols_to_plot].sort_values(by=sort_by).dropna()
fig, axes = vbp.make_multi_cmap_heatmap(
    data_to_plot, 
    heatmap_defs, 
    figsize=(10,12), 
    top_buffer=0, 
    bottom_buffer=0.1, 
    n_cbar_rows=6, 
    heatmap_div=0.9, 
)

<IPython.core.display.Javascript object>

## exclude mesoscope sessions, only include cells matched across sessions

In [23]:
sort_by = [
    'cre_line_categorical',
    'imaging_depth'
]
data_to_plot = (
    dominant_dropout_matrix.query('equipment_name != "MESO.1"')[cols_to_plot]
    .sort_values(by=sort_by)
    .dropna()
)
fig, axes = vbp.make_multi_cmap_heatmap(
    data_to_plot, 
    heatmap_defs, 
    figsize=(10,12), 
    top_buffer=0, 
    bottom_buffer=0.1, 
    n_cbar_rows=5, 
    heatmap_div=0.9, 
)

UndefinedVariableError: name 'equipment_name' is not defined

## sort only by cre-line and depth (still excluding mesoscope)

In [ ]:
sort_by = [
    'cre_line_categorical',
    'imaging_depth',
]
data_to_plot = (
    dominant_dropout_matrix.query('equipment_name != "MESO.1"')[cols_to_plot]
    .sort_values(by=sort_by)
    .dropna()
)
fig, axes = vbp.make_multi_cmap_heatmap(
    data_to_plot, 
    heatmap_defs, 
    figsize=(10,12), 
    top_buffer=0, 
    bottom_buffer=0.1, 
    n_cbar_rows=5, 
    heatmap_div=0.9, 
)

## sankey plots

In [ ]:
import plotly.graph_objects as go
import itertools

In [ ]:
def sankey_diagram(data_to_plot, title_text=''):
    
    # define colormap and categories
    cmap = ['rgb({}, {}, {})'.format(int(255*v[0]),int(255*v[1]),int(255*v[2])) for v in sns.color_palette("Dark2", 4)]
    categories = list(np.sort([c for c in dropout_summary_pivoted['dominant_dropout'].unique() if pd.notnull(c)]))

    # initialize the link dictionary
    link_dict = dict(
        source = [], # indices correspond to labels, eg A1, A2, A2, B1, ...
        target = [],
        value =  [],
        label =  []

    )

    # define values and session IDs
    possible_vals = np.arange(len(categories))
    session_pairs = [(1,3),(3,4),(4,6)]
    # iterate over session pairs:
    for ii, (session_id_1, session_id_2) in enumerate(session_pairs):
        # iterate over all possible combination of dropout values:
        for first_val, second_val in list(itertools.product(possible_vals, possible_vals)):
            # find all rows that meet the condition of having the two sessions equal to the two values
            query = 'session_{} == {} and session_{} == {}'.format(session_id_1, first_val, session_id_2, second_val)
            # update the link dict with relevant data
            link_dict['source'].append(first_val + 4*ii)
            link_dict['target'].append(second_val+ 4*(ii+1))
            link_dict['value'].append(len(data_to_plot.query(query)))
            
            # calculate the fraction of the total, update the label
            label_total = len(data_to_plot.query(query.split(' and')[0]))
            label_fraction = 0 if label_total == 0 else len(data_to_plot.query(query))/label_total
            label = '{} to {} probability: {:0.3f}'.format(categories[first_val], categories[second_val], label_fraction)
            link_dict['label'].append(label)
    
    # add category labels
    category_labels = []
    for session_id in [1,3,4,6]:
        for category_index, category in enumerate(categories):
            query = 'session_{} == {}'.format(session_id, category_index)
            fraction = len(data_to_plot.query(query))/len(data_to_plot)
            category_labels.append('{}\n({:0.1f}%)'.format(category, 100*fraction))

    # define the sankey diagram
    fig = go.Figure(data=[go.Sankey(
        node = dict(
          pad = 15,
          thickness = 20,
          line = dict(color = "black", width = 0.5),
          label = 4*category_labels,
          color = 4*cmap
        ),
        link = link_dict)])

    # update the layout
    fig.update_layout(title_text=title_text, font_size=10)
    
    # return the figure object:
    return fig

In [ ]:
cre_lines = np.sort(dominant_dropout_matrix['cre_line'].unique())
cre_lines

In [ ]:
for cre_line in cre_lines:
    data_to_plot = dominant_dropout_matrix.sort_values(by=sort_by).dropna().query('cre_line == "{}"'.format(cre_line))
    fig = sankey_diagram(data_to_plot, title_text='Sankey plot, GLM verion = {}, cre-line = {}'.format(glm_version, cre_line))
    fig.write_html("/home/dougo/code/dougollerenshaw.github.io/figures_to_share/sankey_diagram_glm_version_{}_cre_line_{}.html".format(glm_version, cre_line))
    fig.show()